# MANU 465 Project: 3D Printing

### Authors:

_Group 7_

Liam Bontkes, 25530163

Stacy Shang

Theophile Calloc'h

Tony Lyu

## 1 Project Description

The goal of this project is to create a machine learning model which is capable of identifying defective print layers
and classifying 3D print layers as Pass/Fail with 85% or greater accuracy. The machine learning model will use a series
of images of print layers (defective and non-defective), interpreted with a machine vision model. To reduce the scope
of the project, we will only train the model on simple geometric shapes such as cubes, spheres and pyramids.
Additionally, we will only be training the model on 3D prints from a fused filament fabrication printer using
polylactic acid filament and a 4mm extruder head.

## 2 Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 3 Data Preprocessing

Preprocess the training and test set.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(rescale=1. / 255,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True)

training_set = image_data_generator.flow_from_directory('data/training_set',
                                                        target_size=(64, 64),
                                                        batch_size=32,
                                                        class_mode='binary')

test_set = image_data_generator.flow_from_directory('data/test_set',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

## 4 Building the CNN Model

### 4.1 Initialize the Model

In [ ]:
model = tf.keras.models.Sequential()

### 4.2 Add Convolutional Layers

In [ ]:
# add and pool 1st layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

In [ ]:
# add and pool 2nd layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### 4.3 Flatten the Model

In [ ]:
model.add(tf.keras.layers.Flatten())

### 4.4 Add Connection Layer

In [ ]:
model.add(tf.keras.layers.Dense(units=256, activation='relu'))

### 4.5 Add Output Layer

In [ ]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### 4.6 Compile the Model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 5 Train the CNN Model

In [ ]:
model.fit(x=training_set, validation_data=test_set, epochs=50)

## 6 Make a Prediction

In [ ]:
from keras.preprocessing import image

prediction_image = image.load_img('data', target_size=(64, 64))
prediction_image = image.img_to_array(prediction_image)
prediction_image = np.expand_dims(prediction_image, axis=0)

result = model.predict(prediction_image)

if result[0][0] == 1:
    prediction = 'Part is OK.'
else:
    prediction = 'Part is defective!'
print(prediction)